---
This file contains the computation needed in Sections 4 and 5 of the paper "Quotients of flag varieties and their birational geometry" by Barban, Occhetta and Solá Conde.

---

# Section 4: Affine charts and their quotients

## Section 4.2. The combinatorial quotient of the Lie algebra of upper triangular matrices.

### Proposition 4.1

We define the map $\pi:\text{N}(T)\to\text{N}(T/H)$, associated to the weight map of the $H$-action on $F_e$.

In [1]:
M = matrix([[-1,-1,0,1,0,0],[0,-1,-1,0,1,0],[-1,-1,-1,0,0,1]])

---

We compute the combinatorial quotient $X_e$ of the affine patch $F_e\simeq \mathbb{C}^6$ under the action of $H$.

We first introduce the affine toric variety corresponding to $F_e$.

---

In [2]:
Fe = AffineToricVariety(cone = Cone(identity_matrix(6).rows()))
ConesFe = [matrix(r.rays()).transpose() for k in srange(1, 6) for r in Fe.fan(k)]

---
We consider the projection of the cones defining $F_e$ under the quotient map $\pi$. 

We remove those which are not strictly convex, and compute the common intersections among the projected ones. We keep only those which have maximal dimension and are not equivalent to existing ones.

---

In [3]:
###################################
### Compute the projected cones ###
###################################

ProjectedCones = [Cone((M*cone).columns()) for cone in ConesFe if Cone((M*cone).columns()).is_strictly_convex()]

##########################################################
### Take common intersection among the projected cones ###
##########################################################

NewCones = [] 
for i in range(len(ProjectedCones)):
	for j in range(i, len(ProjectedCones)):
		c = ProjectedCones[i].intersection(ProjectedCones[j])
		if c.dim() == 3 and not any(c.is_equivalent(d) for d in NewCones):
			NewCones.append(c)

###########################################################
### Remove the redundant cones contained in other cones ###
###########################################################

DeletedCones = set()
for i in range(len(NewCones)):
		if i in DeletedCones:
			continue
		for j in range(i+1, len(NewCones)):
			if j in DeletedCones:
				continue
			ci = NewCones[i]
			cj = NewCones[j]
			cint = ci.intersection(cj)
			if cint.is_equivalent(cj):
				DeletedCones.add(i)
				break
			elif cint.is_equivalent(ci):
				DeletedCones.add(j)

---

The combinatorial quotient fan of $F_e$ by $H$ is defined by the strictly convex projected cones, together with their common intersections.

---

In [4]:
##################################
### Combinatorial quotient fan ###
##################################

QuotientFanCones = [NewCones[i] for i in range(len(NewCones)) if i not in DeletedCones]
QuotientFanRays = set()
for cone in QuotientFanCones:
	QuotientFanRays.update(cone.rays())
QuotientFanRays = list(QuotientFanRays)

QuotientFan= Fan(rays = QuotientFanRays, cones = QuotientFanCones)

######################################
### Combinatorial quotient variety ###
######################################

Xe = ToricVariety(QuotientFan)

##########################################
### Rays of the combinatorial quotient ###
##########################################

Xe.fan().rays()

N( 0, -1, -1),
N( 0,  1,  0),
N(-1,  0, -1),
N(-1, -1, -1),
N( 1,  0,  0),
N( 0,  0, -1),
N( 0,  0,  1)
in 3-d lattice N

---

We check the combinatorial quotient $X_e$ is smooth and has Picard number $4$.

---

In [5]:
Xe.is_smooth(), Xe.rational_class_group().rank()

(True, 4)

### Figure 2


---
The divisor $D = 5C_{02}+3A_1+3B_2+2D_{12}$ is ample, and we compute the associated ample polytope.

---

In [6]:
D = [3,0,3,5,0,2,0]

ieq=(matrix(D).transpose()).augment(matrix(Xe.fan().rays())).rows()

AmplePolytope = Polyhedron(ieqs=ieq)
AmplePolytope.plot()

Graphics3d Object

# Section 5. The tile group

## Section 5.2. Generators of the tile group

### Construction 2

---
We introduce some background functions for Construction $2$, which will be described by the function **birational(perm)** below.

---

In [7]:
######################################################
### Normalize a list of polynomials up to a scalar ###
######################################################

def clear_denominators(poly_list):
    from sage.arith.all import lcm
    denoms = [c.denominator() for f in poly_list for c in f.coefficients()]
    L = lcm(denoms)
    return [L*f for f in poly_list]

##################################################
#####    Normalize a matrix up to a scalar   #####
##################################################

def norm(M):
    entries = [a for a in M.list() if a != 0]
    if not entries:
        return M
    lcm_val = lcm([a.denominator() for a in entries])
    M = M * lcm_val
    gcd_val = gcd([a for a in M.list() if a != 0])
    M = M.apply_map(lambda x: x / gcd_val)
    first_nonzero = next((a for a in M.list() if a != 0), None)
    if first_nonzero and first_nonzero < 0:
        M = -M
    return M

##################################
#### Matrix of a linear map   ####
##################################

def matrix_linearmap(exprs, vars):
    exprs_sym = [SR(e) for e in exprs]  # ensure symbolic form
    rows = []
    for e in exprs_sym:
        coeffs = [e.coefficient(v) for v in vars]
        rows.append(coeffs)
    return Matrix(SR, rows)

##############################################
#####   Simplifies entries of a matrix   #####
##############################################

def simp(M):
    return M.apply_map(lambda x: x.full_simplify())

---
We introduce the symmetric group $S_4$. Then we consider the $H$-orbits of the general elements in $F_e$; in each of these orbits there is a unique element represented by the class of a matrix of the form

$$
Y=\begin{pmatrix}
	1 & 0 & 0 & 0 \\
	1 & 1 & 0 & 0 \\
	y_{1} & 1 & 1 & 0 \\
	y_{3} & y_{2} & 1 & 1 \\
\end{pmatrix}.
$$
Such element corresponds to the point $[1:y_1:y_2:y_3]\in\mathbb{P}^3$.

---

In [14]:
##########################
### Symmetric group S4 ###
##########################

S=SymmetricGroup(list(range(4)))

#####################################
### Polynomial ring in 4 variable ###
#####################################

R.<y0,y1,y2,y3> = PolynomialRing(QQ)
vars=[y0,y1,y2,y3]

#################################
### Lower triangular matrix Y ###
#################################

Y= Matrix(R,[[1,0,0,0],[1,1,0,0],[y1,1,1,0],[y3,y2,1,1]])

In [15]:
Y

[ 1  0  0  0]
[ 1  1  0  0]
[y1  1  1  0]
[y3 y2  1  1]

---
The function **birational(perm)** mimics Construction $2$. Namely, given a permutation **perm** of $S_4$ it computes the rational map $\mathbb{P}^3\dashrightarrow \mathbb{P}^3$ defined by the homogeneous polynomials $[1:f_1:f_2:f_3]$ in the variables $y_0,y_1,y_2,y_3$. 

---

In [145]:
def homogen(L):
    R=L[0].parent()
    Lh=[f.homogenize('y0') for f in L]
    d=max(f.degree() for f in Lh)
    return [f*y0**(d-f.degree()) for f in Lh]

def birational(perm):
    C = (S(perm).matrix().transpose())*Y
    P,L,U = C.LU()
    MH = [L[1,0]*L[2,1]*L[3,2], L[3,2]*L[2,0], L[1,0]*L[3,1], L[3,0]] 
    return homogen([R(f) for f in MH])

---
Finally we introduce the change of coordinates (7), that will make the maps $r_i$, $\tau$ acquire the expressions described in Proposition 5.3. 

---

In [146]:
#############################
### Change of coordinates ###
#############################

Frac = R.fraction_field()
CC = Matrix(Frac,[(1, 0, 0, 0),(1, -1, 0, 0), (0, 0, 1, 0), (0, 0, 1, -1)]) 

### Proposition 5.3

---

We compute the matrix associated to the rational maps defined by the reflections $r_1,r_2,r_3$ and the anti-transposition $\tau$.

Reflection $r_1$.

---

In [147]:
#####################
### Reflection R1 ###
#####################

r1=matrix_linearmap(birational([1,0,2,3]),vars)
R1=norm(CC*r1*CC.inverse()); R1

[0 1 0 0]
[1 0 0 0]
[0 0 0 1]
[0 0 1 0]

---

Reflection $r_2$.

---

In [148]:
r2=birational([0,2,1,3])

#############################
### Change of coordinates ###
#############################

var('z0 z1 z2 z3')
z = vector(SR, [z0,z1,z2,z3])
y = CC.inverse() * z

subs_map = {R.gens()[i]: y[i] for i in range(4)}
evaluated = [f.subs(subs_map).simplify_full() for f in r2]

F = CC * vector(evaluated)
[poly.simplify_full() for poly in F]

[-z0^2 + z0*z1 + (z0 - z1)*z2, z0*z1 - z1*z2, (z0 - z1)*z2, -z1*z2 + z0*z3]

In [149]:
####################################
### Cremona map associated to R2 ###
####################################

def cremona(A):
    M=Matrix([z0,0,0,0]).transpose()
    M[0,0]= (A[1,0]-A[0,0]) * (A[2,0] - A[0,0])
    M[1,0]= A[1,0] * (A[2,0] - A[0,0])
    M[2,0]= A[2,0] * (A[1,0] - A[0,0])
    M[3,0]= A[1,0] * A[2,0]- A[0,0] * A[3,0] 
    return norm(M)

---

Reflection $r_3$.

---

In [150]:
r3=matrix_linearmap(birational([0,1,3,2]),vars)
R3=norm(CC*r3*CC.inverse()); R3

[0 0 1 0]
[0 0 0 1]
[1 0 0 0]
[0 1 0 0]

---

Reflection $\tau$. We adapt Construction $2$ to this specific case.

---

In [151]:
#############################################
### Construction 2 for anti-transposition ###
#############################################

perm=([3,2,1,0])                       
C = S([3,2,1,0]).matrix()*(Y.transpose().inverse())*S([3,2,1,0]).matrix()                                   
P,L,U = C.LU()                                                                    
MH = [L[1,0]*L[2,1]*L[3,2], L[3,2]*L[2,0], L[1,0]*L[3,1], L[3,0]]
homogen([R(f) for f in MH])

########################################
### Matrix of the anti-transposition ###
########################################

tau=matrix_linearmap(homogen([R(f) for f in MH]), vars)
T=norm(CC*tau*CC.inverse()); T

[1 0 0 0]
[0 0 1 0]
[0 1 0 0]
[0 0 0 1]

### Relations among $r_1,r_2,r_3$ and $\tau$.

---

We check all generators are involutions.

---

In [152]:
#########################
### Check involutions ###
#########################

ZZ=Matrix([z0,z1,z2,z3]).transpose()

norm(R1*R1), simp(cremona(cremona(ZZ))), norm(R3*R3), norm(T*T)

(
[1 0 0 0]  [-z0]  [1 0 0 0]  [1 0 0 0]
[0 1 0 0]  [-z1]  [0 1 0 0]  [0 1 0 0]
[0 0 1 0]  [-z2]  [0 0 1 0]  [0 0 1 0]
[0 0 0 1], [-z3], [0 0 0 1], [0 0 0 1]
)

---
We check that the relations among the generators are exactly those of the octahedral group.

---

In [153]:
###################################
### R1 * R2 * R1 = R2 * R1 * R2 ###
###################################

simp(R1*cremona(R1*ZZ)), simp(cremona(R1*cremona(ZZ)))

(
[              -z0*z1 + z0*z3]  [              z0*z1 - z0*z3]
[-z0*z1 + z1^2 + (z0 - z1)*z3]  [z0*z1 - z1^2 - (z0 - z1)*z3]
[              -z1*z2 + z0*z3]  [              z1*z2 - z0*z3]
[                (z0 - z1)*z3], [              -(z0 - z1)*z3]
)

In [154]:
###################################
### R3 * R2 * R3 = R2 * R3 * R2 ###
###################################

simp(R3*cremona(R3*ZZ)), simp(cremona(R3*cremona(ZZ)))

(
[              -z0*z2 + z0*z3]  [              z0*z2 - z0*z3]
[              -z1*z2 + z0*z3]  [              z1*z2 - z0*z3]
[-z0*z2 + z2^2 + (z0 - z2)*z3]  [z0*z2 - z2^2 - (z0 - z2)*z3]
[                (z0 - z2)*z3], [              -(z0 - z2)*z3]
)

In [155]:
#######################
### T * R1 * T = R3 ###
#######################

T*R1*T-R3

[0 0 0 0]
[0 0 0 0]
[0 0 0 0]
[0 0 0 0]

In [144]:
#######################
### T * R2 * T = R2 ###
#######################

T*cremona(T*ZZ)-cremona(ZZ)

[0]
[0]
[0]
[0]

## Section 5.3. The boundary divisors

### Corollary 5.10

---
We introduce the divisors $A_1, B_2, C_{02}, D_{12}$.

---

In [110]:
A1=matrix([0,y1,0,y3]).transpose()
B2=matrix([0,0,y2,y3]).transpose() 
C02=matrix([0,y1,y2,y3]).transpose() 
D12=matrix([0, 0, 0, 1]).transpose()  

---
We apply $r_1$ and $r_3$ to $A_1, B_2, C_{02}, D_{12}$.

---

In [111]:
A0 = R1*A1; A0

[y1]
[ 0]
[y3]
[ 0]

In [112]:
B3=R3*B2; B3

[y2]
[y3]
[ 0]
[ 0]

In [113]:
C12 = R1*C02; C12

[y1]
[ 0]
[y3]
[y2]

In [114]:
C03 = R3*C02; C03

[y2]
[y3]
[ 0]
[y1]

In [115]:
C13= R1*R3*C02; C13

[y3]
[y2]
[y1]
[ 0]

In [116]:
D02=R1*D12; D02

[0]
[0]
[1]
[0]

In [117]:
D13= R3*D12; D13 

[0]
[1]
[0]
[0]

In [118]:
D03= R1*R3*D12; D03 

[1]
[0]
[0]
[0]

---
We find $A_2,B_1,D_{01}$ and $C_{23}$ by looking at their images via $r_2$.

---

In [119]:
A2=matrix([y0,y0,y2,y3]).transpose() 
cremona(A2)

[                 0]
[-(y0^2 - y0*y2)/y0]
[                 0]
[(y0*y2 - y0*y3)/y0]

In [120]:
B1=matrix([y0,y1,y0,y3]).transpose()
cremona(B1)

[                 0]
[                 0]
[-(y0^2 - y0*y1)/y0]
[(y0*y1 - y0*y3)/y0]

In [121]:
D01= matrix([y0,y1,y0,y1]).transpose()
cremona(D01)

[0]
[0]
[1]
[0]

In [122]:
C23 = matrix([y0*y2,y0*y3,y1*y2,y1*y3]).transpose(); 
cremona(C23)

[(y0^2*y2^2 - y0*y1*y2^2 - y0^2*y2*y3 + y0*y1*y2*y3)/(y0*y2)]
[                        -(y0^2*y2*y3 - y0*y1*y2*y3)/(y0*y2)]
[                        -(y0*y1*y2^2 - y0*y1*y2*y3)/(y0*y2)]
[                                                          0]

---
The remaining divisors are obtained by applying specific reflections to certain divisors.

---

In [123]:
B0=R1*B1; B0

[y1]
[y0]
[y3]
[y0]

In [124]:
C01=cremona(C02); C01

[1]
[1]
[1]
[1]

In [125]:
A3=R3*A2; A3

[y2]
[y3]
[y0]
[y0]

In [41]:
D23 = T*D01; D23

[y0]
[y0]
[y1]
[y1]